In [1]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model

actions = ['come', 'away', 'spin']
seq_length = 30

model = load_model('/Users/jeongtaewon/PycharmProjects/gesture-recognition/models/model2.h5')

# MediaPipe hands model
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=2,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5)

cap = cv2.VideoCapture(0)

# w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
# h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
# fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
# out = cv2.VideoWriter('input.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), (w, h))
# out2 = cv2.VideoWriter('output.mp4', fourcc, cap.get(cv2.CAP_PROP_FPS), (w, h))

seq = []
action_seq = []

while cap.isOpened():
    ret, img = cap.read()
    img0 = img.copy()

    img = cv2.flip(img, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    result = hands.process(img)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    if result.multi_hand_landmarks is not None:
        for res in result.multi_hand_landmarks:
            joint = np.zeros((21, 4))
            for j, lm in enumerate(res.landmark):
                joint[j] = [lm.x, lm.y, lm.z, lm.visibility]

            # Compute angles between joints
            v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19], :3] # Parent joint
            v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20], :3] # Child joint
            v = v2 - v1 # [20, 3]
            # Normalize v
            v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

            # Get angle using arcos of dot product
            angle = np.arccos(np.einsum('nt,nt->n',
                v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

            angle = np.degrees(angle) # Convert radian to degree

            d = np.concatenate([joint.flatten(), angle])

            seq.append(d)

            mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)

            if len(seq) < seq_length:
                continue

            input_data = np.expand_dims(np.array(seq[-seq_length:], dtype=np.float32), axis=0)

            y_pred = model.predict(input_data).squeeze()
            # 결과를 뽑아내서 y_pred에 저장
            i_pred = int(np.argmax(y_pred))
            # 어떤인덱스인지 , conf를 뽑아낸다.
            conf = y_pred[i_pred]

            if conf < 0.9:
                continue
            # 0.9보다 낮다면 제스처를 취하지 않았다고 인지하고 버린다.
            action = actions[i_pred]
            action_seq.append(action)
            #seq에 저장한다.

            if len(action_seq) < 3:
                continue

            this_action = '?'
            if action_seq[-1] == action_seq[-2] == action_seq[-3]:
                this_action = action

            #마지막 3개의 액션이 모두 같다면 유효한 액션으로 판단후 화면에 띄운다. 그렇지 않다면 "?"를 출력.

            cv2.putText(img, f'{this_action.upper()}', org=(int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)

    # out.write(img0)
    # out2.write(img)
    cv2.imshow('img', img)
    if cv2.waitKey(1) == ord('q'):
        break

2023-01-10 23:32:38.043136: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-10 23:32:38.043739: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 8. Tune using inter_op_parallelism_threads for best performance.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


KeyboardInterrupt: 

In [1]:
import sys

 

print(sys.version)

3.7.15 (default, Nov 10 2022, 12:46:26) 
[Clang 14.0.6 ]


In [2]:
pip uninstall numpy -y

Found existing installation: numpy 1.21.5
Uninstalling numpy-1.21.5:
  Successfully uninstalled numpy-1.21.5
Note: you may need to restart the kernel to use updated packages.


In [ ]:
conda update --all

Solving environment: done

## Package Plan ##

  environment location: /Users/jeongtaewon/opt/anaconda3


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _anaconda_depends-2022.05  |           py39_0           7 KB
    aiohttp-3.8.3              |   py39h6c40b1e_0         428 KB
    aiosignal-1.2.0            |     pyhd3eb1b0_0          12 KB
    anaconda-custom            |           py39_1           4 KB
    anaconda-navigator-2.3.2   |   py39hecd8cb5_0         6.6 MB
    arrow-1.2.3                |   py39hecd8cb5_0         146 KB
    asttokens-2.0.5            |     pyhd3eb1b0_0          20 KB
    async-timeout-4.0.2        |   py39hecd8cb5_0          13 KB
    attrs-22.1.0               |   py39hecd8cb5_0          84 KB
    babel-2.11.0               |   py39hecd8cb5_0         6.8 MB
    botocore-1.27.59           |   py39hecd8cb5_0         5.6 MB
    cachetools-4.2.2           |   

scipy-1.7.3          | 15.8 MB   | ##################################### | 100% 
jupyter_server-1.23. | 387 KB    | ##################################### | 100% 
libedit-3.1.20221030 | 155 KB    | ##################################### | 100% 
fsspec-2022.11.0     | 224 KB    | ##################################### | 100% 
pkginfo-1.8.3        | 47 KB     | ##################################### | 100% 
cfitsio-3.470        | 782 KB    | ##################################### | 100% 
hdf5-1.10.6          | 3.0 MB    | ##################################### | 100% 
google-cloud-core-2. | 39 KB     | ##################################### | 100% 
greenlet-2.0.1       | 184 KB    | ##################################### | 100% 
libtiff-4.4.0        | 504 KB    | ##################################### | 100% 
distributed-2022.2.1 | 659 KB    | ##################################### | 100% 
anaconda-navigator-2 | 6.6 MB    | ##################################### | 100% 
_anaconda_depends-20 | 7 KB 

In [10]:
apt-get install protobuf-compiler

SyntaxError: invalid syntax (925973800.py, line 1)

In [3]:
pip install opencv-python

  Using cached opencv_python-4.7.0.68-cp37-abi3-macosx_10_13_x86_64.whl (51.7 MB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install mediapipe

  Using cached mediapipe-0.9.0.1-cp37-cp37m-macosx_10_15_x86_64.whl (35.2 MB)
  Using cached opencv_contrib_python-4.7.0.68-cp37-abi3-macosx_10_13_x86_64.whl (61.3 MB)
  Using cached absl_py-1.3.0-py3-none-any.whl (124 kB)
  Using cached flatbuffers-22.12.6-py2.py3-none-any.whl (26 kB)
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install tensorflow

  Using cached tensorflow-2.11.0-cp37-cp37m-macosx_10_14_x86_64.whl (244.2 MB)
  Using cached tensorflow_estimator-2.11.0-py2.py3-none-any.whl (439 kB)
  Using cached termcolor-2.1.1-py3-none-any.whl (6.2 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached tensorflow_io_gcs_filesystem-0.29.0-cp37-cp37m-macosx_10_14_x86_64.whl (1.6 MB)
  Using cached libclang-14.0.6-py2.py3-none-macosx_10_9_x86_64.whl (13.2 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached keras-2.11.0-py2.py3-none-any.whl (1.7 MB)
  Using cached protobuf-3.19.6-cp37-cp37m-macosx_10_9_x86_64.whl (979 kB)
  Using cached tensorboard-2.11.0-py3-none-any.whl (6.0 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-macosx_10_9_x86_64.whl (3.5 MB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_plugin_wit-1.8

In [1]:
conda install numpy=1.19.5 -c conda-forge

Solving environment: | 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/osx-64::_anaconda_depends==2022.05=py37_0
  - defaults/noarch::tifffile==2021.7.2=pyhd3eb1b0_2
  - defaults/noarch::opt_einsum==3.3.0=pyhd3eb1b0_1
  - conda-forge/osx-64::gensim==4.2.0=py37h233bc55_0
  - defaults/osx-64::datashape==0.5.4=py37hecd8cb5_1
  - defaults/osx-64::google-cloud-core==2.3.2=py37hecd8cb5_0
  - defaults/osx-64::scikit-learn==1.0.2=py37hae1ba45_1
  - defaults/osx-64::imageio==2.19.3=py37hecd8cb5_0
  - defaults/noarch::keras-preprocessing==1.1.2=pyhd3eb1b0_0
  - defaults/osx-64::holoviews==1.15.3=py37hecd8cb5_0
  - defaults/osx-64::bottleneck==1.3.5=py37h67323c0_0
  - defaults/osx-64::panel==0.14.1=py37hecd8cb5_0
  - defaults/osx-64::googleapis-common-protos==1.56.4=py37hecd8cb5_0
  - defaults/osx-64::contourpy==1.0.5=py37haf03e11_0
  - defaults/osx-64::matplotlib==3.5.3=py37hecd8cb5_0
  - defaults/osx-6

In [3]:
pip uninstall numpy -y

Found existing installation: numpy 1.19.5
Uninstalling numpy-1.19.5:
  Successfully uninstalled numpy-1.19.5
Note: you may need to restart the kernel to use updated packages.


In [ ]:
conda install numpy=1.20.3 -c conda-forge

In [2]:
pip install numpy==1.19.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 2.7 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.20.3
    Uninstalling numpy-1.20.3:
      Successfully uninstalled numpy-1.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
tensorflow 2.11.0 requires numpy>=1.20, but you have numpy 1.19.5 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
conda list

# packages in environment at /Users/jeongtaewon/opt/anaconda3:
#
# Name                    Version                   Build  Channel
_anaconda_depends         2022.05                  py37_0  
_ipyw_jlab_nb_ext_conf    0.1.0            py37hecd8cb5_1  
_tflow_select             2.3.0                       mkl  
absl-py                   1.3.0            py37hecd8cb5_0  
aiohttp                   3.8.3            py37h6c40b1e_0  
aiosignal                 1.2.0              pyhd3eb1b0_0  
alabaster                 0.7.12             pyhd3eb1b0_0  
anaconda                  custom                   py37_1  
anaconda-client           1.11.0           py37hecd8cb5_0  
anaconda-navigator        2.3.2            py37hecd8cb5_0  
anaconda-project          0.11.1           py37hecd8cb5_0  
anyio                     3.5.0            py37hecd8cb5_0  
appdirs                   1.4.4              pyhd3eb1b0_0  
applaunchservices         0.3.0            py37hecd8cb5_0  
appnope                   0.